# Zeroth Order Optimization (ZOO) Attack Experiments - Zoo-DoH
## Warning: Before running this notebook, make sure you normalized the features (normalize-data.ipynb) and build the model (build_model.ipynb)


In [ ]:
import numpy as np
import pandas as pd

from joblib import dump, load

#!pip install adversarial-robustness-toolbox
from art.estimators.classification import SklearnClassifier

# Loading Dataset

In [ ]:
input_dir = '../datasets/'

In [ ]:
X_features_dnstt_file = input_dir + '27072024-tunnel.csv'

In [ ]:
from sklearn.preprocessing import normalize

# Load the X_features 
X_features_dnstt = pd.read_csv(X_features_dnstt_file, sep=',')

# concat all features
X = pd.concat([X_features_dnstt])
X = X.drop(columns=['SourceIP', 'DestinationIP', 'TimeStamp', 'SourcePort', 'DestinationPort', 'Duration', 'DoH'])

display(X)

In [ ]:
# Normalize the features
X = pd.DataFrame(normalize(X, norm='l2', axis=1), columns=X.columns)
display(X.head())

# Attacking

In [ ]:
model_input_dir = './'

# Evaluating Constrained Zoo-DoH - DNSTT - GradientBoosting

In [ ]:
# Load the X_features
# Convert the DataFrame to a NumPy array
X_test = np.array(X)

In [ ]:
display(X_test.shape[0])

In [ ]:
import myzoo.zoo_doh as zoo_doh

model_filename = 'GradientBoosting-e-valente-customized.joblib'
model = load(model_input_dir + model_filename)

# Create blackbox object
art_classifier = SklearnClassifier(model=model)

# Create ART Zeroth Order Optimization attack
zoo = zoo_doh.ZooAttack(classifier=art_classifier, confidence=0.0, targeted=True, learning_rate=1e-2, max_iter=20,
                    binary_search_steps=10, initial_const=1e-3, abort_early=True, use_importance=False, nb_parallel=1, 
                        batch_size=1, variable_h=0.2)

In [ ]:
# attacking
size_data = len(X_test)
x_test_adv = zoo.generate(X_test, np.zeros(size_data, dtype='int'))

In [ ]:
# attack success rate
success = 0
total_to_attack = 0

success_indices = []
for i in range(size_data):
  prediction_before_attack = model.predict(X_test[i].reshape(1, -1))
  prediction_after_attack = model.predict(x_test_adv[i].reshape(1, -1))

  if prediction_before_attack == [1]:
    total_to_attack = total_to_attack + 1

    if prediction_after_attack == [0]:
      success = success + 1
      success_indices.append(i)
  
print(f'total samples to be attacked (malicious samples) {total_to_attack}')

print(f'total success (malicious -> benign) {success}')
print(f'total percent samples successfully attacked {success/total_to_attack}')